In [ ]:
#pip install entsoe-py

Prices are in MW (Mega Watt), which equals to 1000 kW or 1.000.000 W. Therefore the generated table gets divided by 1.000.000 to get the price for one W.

# Testing the individual steps

In [53]:
date = "2023-03-06"
timezone = 'Europe/Brussels'

In [54]:
import pandas as pd
from datetime import datetime, timedelta
from entsoe import EntsoePandasClient
import pytz
date = pd.to_datetime(date,format= '%Y-%m-%d')
# looking for tommorow prices
date = date + timedelta(days = 1)
current_timezone = pytz.timezone(timezone)
date = current_timezone.localize(date)

In [55]:
start = (date - timedelta(days= 3)).normalize()
end = (date + timedelta(days = 3)).normalize()
country_code = 'DE_LU'

In [56]:
client = EntsoePandasClient(api_key='6f67ccf4-edb3-4100-a850-969c73688627')
df = client.query_day_ahead_prices(country_code = country_code, start = start, end = end)
df

2023-03-04 00:00:00+01:00    108.27
2023-03-04 01:00:00+01:00     95.86
2023-03-04 02:00:00+01:00     92.18
2023-03-04 03:00:00+01:00     97.96
2023-03-04 04:00:00+01:00     97.69
                              ...  
2023-03-07 19:00:00+01:00    143.91
2023-03-07 20:00:00+01:00    152.14
2023-03-07 21:00:00+01:00    136.43
2023-03-07 22:00:00+01:00    122.21
2023-03-07 23:00:00+01:00    115.49
Freq: 60T, Length: 96, dtype: float64

In [57]:
# handling problem with missing price data for more than 24 hours ahead
if(date.strftime('%Y-%m-%d') == max(df.index).strftime('%Y-%m-%d')):
    date_48 = date + timedelta(days=1)
    for hour in range(24):
        dt = date_48.replace(hour=hour, minute=0, second=0, microsecond=0)
        # Get the price from the day before at this hour
        day_before = dt - timedelta(days=1)
        value = df.loc[day_before]
        # Append the new row to the series
        df.loc[dt] = value
if(date > max(df.index)):
    date_48 = date + timedelta(days=1)
    for hour in range(24):
        dt = date_48.replace(hour=hour, minute=0, second=0, microsecond=0)
        # Get the price from the day before at this hour
        day_before = dt - timedelta(days=1)
        value = df.loc[day_before]
        # Append the new row to the series
        df.loc[dt] = value
    date_48 = date + timedelta(days=2)
    for hour in range(24):
        dt = date_48.replace(hour=hour, minute=0, second=0, microsecond=0)
        # Get the price from the day before at this hour
        day_before = dt - timedelta(days=1)
        value = df.loc[day_before]
        # Append the new row to the series
        df.loc[dt] = value

C:\Users\chris\AppData\Local\Temp\ipykernel_804\2482555799.py:10: FutureWarning: Setitem-like behavior with mismatched timezones is deprecated and will change in a future version. Instead of raising (or for Index, Series, and DataFrame methods, coercing to object dtype), the value being set (or passed as a fill_value, or inserted) will be cast to the existing DatetimeArray/DatetimeIndex/Series/DataFrame column's timezone. To retain the old behavior, explicitly cast to object dtype before the operation.
  df.loc[dt] = value


In [58]:
range_hours = pd.date_range(start=date, freq="H", periods=48)
df = df.loc[range_hours]

In [59]:
df

2023-03-07 00:00:00+01:00     98.06
2023-03-07 01:00:00+01:00     94.26
2023-03-07 02:00:00+01:00     85.00
2023-03-07 03:00:00+01:00     78.96
2023-03-07 04:00:00+01:00     78.52
2023-03-07 05:00:00+01:00     85.57
2023-03-07 06:00:00+01:00     98.71
2023-03-07 07:00:00+01:00     99.57
2023-03-07 08:00:00+01:00    105.59
2023-03-07 09:00:00+01:00     91.92
2023-03-07 10:00:00+01:00     93.86
2023-03-07 11:00:00+01:00     94.26
2023-03-07 12:00:00+01:00     87.57
2023-03-07 13:00:00+01:00     93.86
2023-03-07 14:00:00+01:00     95.07
2023-03-07 15:00:00+01:00     97.60
2023-03-07 16:00:00+01:00    109.22
2023-03-07 17:00:00+01:00    142.97
2023-03-07 18:00:00+01:00    164.08
2023-03-07 19:00:00+01:00    143.91
2023-03-07 20:00:00+01:00    152.14
2023-03-07 21:00:00+01:00    136.43
2023-03-07 22:00:00+01:00    122.21
2023-03-07 23:00:00+01:00    115.49
2023-03-08 00:00:00+01:00     98.06
2023-03-08 01:00:00+01:00     94.26
2023-03-08 02:00:00+01:00     85.00
2023-03-08 03:00:00+01:00   

# Whole price agent

In [38]:
###################################################################################################
# price agent #####################################################################################
###################################################################################################
class Price_Agent(): 

    def return_day_ahead_prices(self, date, timezone = 'Europe/Brussels'):
        import pandas as pd
        from datetime import datetime, timedelta
        from entsoe import EntsoePandasClient
        import pytz
        date = pd.to_datetime(date,format= '%Y-%m-%d')
        # looking for tommorow prices
        date = date + timedelta(days = 1)
        current_timezone = pytz.timezone(timezone)
        date = current_timezone.localize(date)
        start = (date - timedelta(days= 3)).normalize()
        end = (date + timedelta(days = 3)).normalize()
        country_code = 'DE_LU'
        client = EntsoePandasClient(api_key='6f67ccf4-edb3-4100-a850-969c73688627')
        df = client.query_day_ahead_prices(country_code = country_code, start = start, end = end)
        
        # handling problem with missing price data for more than 24 hours ahead
        if(date.strftime('%Y-%m-%d') == max(df.index).strftime('%Y-%m-%d')):
            date_48 = date + timedelta(days=1)
            for hour in range(24):
                dt = date_48.replace(hour=hour, minute=0, second=0, microsecond=0)
                # Get the price from the day before at this hour
                day_before = dt - timedelta(days=1)
                value = df.loc[day_before]
                # Append the new row to the series
                df.loc[dt] = value
        if(date > max(df.index)):
            date_48 = date + timedelta(days=1)
            for hour in range(24):
                dt = date_48.replace(hour=hour, minute=0, second=0, microsecond=0)
                # Get the price from the day before at this hour
                day_before = dt - timedelta(days=1)
                value = df.loc[day_before]
                # Append the new row to the series
                df.loc[dt] = value
            date_48 = date + timedelta(days=2)
            for hour in range(24):
                dt = date_48.replace(hour=hour, minute=0, second=0, microsecond=0)
                # Get the price from the day before at this hour
                day_before = dt - timedelta(days=1)
                value = df.loc[day_before]
                # Append the new row to the series
                df.loc[dt] = value
                
        range_hours = pd.date_range(start=date, freq="H", periods=48)
        df = df.loc[range_hours]
        return df

# Testing the Agent

In [39]:
date = "2023-03-06"

In [40]:
PA = Price_Agent()
price_df = PA.return_day_ahead_prices(date)

C:\Users\chris\AppData\Local\Temp\ipykernel_804\3337783136.py:31: FutureWarning: Setitem-like behavior with mismatched timezones is deprecated and will change in a future version. Instead of raising (or for Index, Series, and DataFrame methods, coercing to object dtype), the value being set (or passed as a fill_value, or inserted) will be cast to the existing DatetimeArray/DatetimeIndex/Series/DataFrame column's timezone. To retain the old behavior, explicitly cast to object dtype before the operation.
  df.loc[dt] = value


In [41]:
price_df

2023-03-07 00:00:00+01:00     98.06
2023-03-07 01:00:00+01:00     94.26
2023-03-07 02:00:00+01:00     85.00
2023-03-07 03:00:00+01:00     78.96
2023-03-07 04:00:00+01:00     78.52
2023-03-07 05:00:00+01:00     85.57
2023-03-07 06:00:00+01:00     98.71
2023-03-07 07:00:00+01:00     99.57
2023-03-07 08:00:00+01:00    105.59
2023-03-07 09:00:00+01:00     91.92
2023-03-07 10:00:00+01:00     93.86
2023-03-07 11:00:00+01:00     94.26
2023-03-07 12:00:00+01:00     87.57
2023-03-07 13:00:00+01:00     93.86
2023-03-07 14:00:00+01:00     95.07
2023-03-07 15:00:00+01:00     97.60
2023-03-07 16:00:00+01:00    109.22
2023-03-07 17:00:00+01:00    142.97
2023-03-07 18:00:00+01:00    164.08
2023-03-07 19:00:00+01:00    143.91
2023-03-07 20:00:00+01:00    152.14
2023-03-07 21:00:00+01:00    136.43
2023-03-07 22:00:00+01:00    122.21
2023-03-07 23:00:00+01:00    115.49
2023-03-08 00:00:00+01:00     98.06
2023-03-08 01:00:00+01:00     94.26
2023-03-08 02:00:00+01:00     85.00
2023-03-08 03:00:00+01:00   